In [1]:
from keras.applications.vgg16 import VGG16
import numpy as np

Using TensorFlow backend.


In [2]:
model=VGG16(include_top=True, weights='imagenet')

In [1]:
from keras.preprocessing import image

path='../dogs_cats_dataset/mini_dogscats/train/cat/'
img_path = 'cat.0.jpg'
img = image.load_img(path+img_path, target_size=(224, 224))

Using TensorFlow backend.


In [4]:
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [5]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [6]:
x.shape

(1, 224, 224, 3)

In [7]:
x = preprocess_input(x)

In [8]:
a=model.predict(x)

In [9]:
a.shape

(1, 1000)

## Action Plan¶
1. Construct a linear model, with take (\*,1000)-array as input
2. Calculate the probability by predict from VGG16(include train, valid and test)
3. Save the result by bcolz.carray
4. Train the linear model
5. Select by validate set
6. Submit predictions to Kaggle

### Add a layer at the end of VGG16

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

In [102]:
lm=Sequential()
lm.add(Dense(1, input_shape=(1000,)))

In [103]:
lm.compile(optimizer=SGD(lr=0.05), loss='mse')

### Calculate the probability by predict from VGG16(include train, valid and test)
1. Generate validate and training data flow

In [104]:
import os, sys
current_dir=os.getcwd()
data_home_dir=current_dir+'\\dogscats\\'

In [105]:
from keras.preprocessing import image

In [106]:
gen=image.ImageDataGenerator()
image_shape=(224,224)
train_batch=gen.flow_from_directory(data_home_dir+'train', batch_size=16, target_size=image_shape)
valid_batch=gen.flow_from_directory(data_home_dir+'valid', batch_size=16, target_size=image_shape)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


2.Generate (n, 1000) vectors

In [82]:
train_input=model.predict_generator(train_batch, np.ceil(train_batch.n/train_batch.batch_size).astype(int) )
valid_input=model.predict_generator(valid_batch, np.ceil(valid_batch.n/valid_batch.batch_size).astype(int) )

In [95]:
valid_input.shape

(2000, 1000)

### Save the result by bcolz.carray

In [85]:
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

In [86]:
save_array(data_home_dir+'train_data.bc', train_input)
save_array(data_home_dir+'valid_data.bc', valid_input)

### Train the linear model

In [107]:
lm.fit(train_input, train_batch.classes, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
23000/23000 [==============================] - 1s - loss: 0.2524     
Epoch 2/10
23000/23000 [==============================] - 1s - loss: 0.2505     
Epoch 3/10
23000/23000 [==============================] - 1s - loss: 0.2502     
Epoch 4/10
23000/23000 [==============================] - 1s - loss: 0.2501     
Epoch 5/10
23000/23000 [==============================] - 1s - loss: 0.2500     
Epoch 6/10
23000/23000 [==============================] - 1s - loss: 0.2500     
Epoch 7/10
23000/23000 [==============================] - 1s - loss: 0.2499     
Epoch 8/10
23000/23000 [==============================] - 1s - loss: 0.2499     
Epoch 9/10
23000/23000 [==============================] - 1s - loss: 0.2498     
Epoch 10/10
23000/23000 [==============================] - 1s - loss: 0.2496     


In [108]:
valid_prediction=lm.predict(valid_input)

In [109]:
valid_prediction.shape

(2000, 1)

In [110]:
valid_prediction[:5]

array([[ 0.49535874],
       [ 0.51686692],
       [ 0.48054034],
       [ 0.51013505],
       [ 0.49001968]], dtype=float32)

In [111]:
lm.evaluate(valid_input,valid_batch.classes)

1248/2000 [=================>............] - ETA: 0s

0.25146267914772036

In [112]:
lm.get_weights()

[array([[  2.71122567e-02],
        [ -7.21198544e-02],
        [ -2.60601230e-02],
        [ -2.72159148e-02],
        [  7.27059469e-02],
        [  7.52517805e-02],
        [  2.13541929e-02],
        [  7.04243407e-02],
        [ -2.48350464e-02],
        [  7.49150873e-04],
        [  5.31235225e-02],
        [ -2.06498448e-02],
        [  6.86585978e-02],
        [ -1.59303530e-03],
        [ -6.64261878e-02],
        [ -3.96760330e-02],
        [ -4.60700244e-02],
        [ -5.59851788e-02],
        [ -1.47002703e-02],
        [ -7.27487132e-02],
        [  3.59616131e-02],
        [  4.11503613e-02],
        [ -5.17554767e-03],
        [  1.61031112e-02],
        [ -5.14333285e-02],
        [  5.10392152e-02],
        [  2.69559901e-02],
        [  4.63074893e-02],
        [  2.73427963e-02],
        [  4.19430137e-02],
        [ -2.31444053e-02],
        [ -5.98447584e-02],
        [  5.88027276e-02],
        [ -3.55621502e-02],
        [  2.47526765e-02],
        [  5.3371978

### Generate results

In [126]:
test_batch=gen.flow_from_directory(data_home_dir+'test1', class_mode=None, batch_size=16, target_size=image_shape)

Found 12500 images belonging to 1 classes.


In [129]:
test_input=model.predict_generator(test_batch, np.ceil(test_batch.n/test_batch.batch_size).astype(int) )

In [132]:
save_array(data_home_dir+'test_data.bc', test_input)

In [130]:
prediction=lm.predict(test_input)

In [131]:
prediction[:5]

array([[ 0.53985512],
       [ 0.46890348],
       [ 0.50125498],
       [ 0.48138046],
       [ 0.49643484]], dtype=float32)

In [170]:
isdog=(prediction>0.5).ravel()

In [169]:
result_label=np.ones(prediction.shape, dtype=np.float)
result_label[:]=0.05
result_label[isdog]=0.95
result_label[:5]

array([[ 0.95],
       [ 0.05],
       [ 0.95],
       [ 0.05],
       [ 0.05]])

### Save result

In [171]:
ids=np.arange(len(result_label)).reshape((-1,1))
subm=np.concatenate([ids, result_label.reshape(-1,1)],axis=1)

In [173]:
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')